# Z-score Testing 
**By Ibrahim Youssef**

This is a tool made to test the change using the Z-score will give it a certain date and time 
this is used to test differences in the metrics
this is used to test the difference in a certain metric like Sales and conversion

# Getting the Tool kits

In [1]:
import pandas as pd 
import numpy as np 
import warnings 

In [2]:
warnings.filterwarnings("ignore")

# Getting the data
for example, we will use this data base and do all the cleaning for it before running the test on what we need

In [3]:
url='https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
data=pd.read_excel(url)
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


# Data Cleaning and preparation 

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


## What are the columns with missing values?

In [5]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Removing cancelled orders 

In [6]:
data=data.loc[data['InvoiceNo'].str.contains('C')!=True]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532621 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    532621 non-null  object        
 1   StockCode    532621 non-null  object        
 2   Description  531167 non-null  object        
 3   Quantity     532621 non-null  int64         
 4   InvoiceDate  532621 non-null  datetime64[ns]
 5   UnitPrice    532621 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      532621 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.6+ MB


## Adding a total price column  

In [7]:
data['Total_price']=data['Quantity']*data['UnitPrice']

## Checking the max and min times 

In [8]:
import datetime as dt
from datetime import timedelta

In [9]:
data['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [10]:
data['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [11]:
print('The Range between the first and the last day is',data['InvoiceDate'].max()-data['InvoiceDate'].min())

The Range between the first and the last day is 373 days 04:24:00


## dates for the test

Lets get the 

In [12]:
from dateutil.relativedelta import relativedelta  
date_check=dt.datetime(2011,6,12) # add date time for the change as follows year,month,day
b_a_time=30 #add the date time for the test of befor and after 
first_date=date_check-timedelta(days=b_a_time)
last_date=date_check+ timedelta(days=b_a_time)
print('test date',date_check ,'\nfirst date',first_date,'\nlast date',last_date,'\noffset days from the test date',b_a_time,'Days')

test date 2011-06-12 00:00:00 
first date 2011-05-13 00:00:00 
last date 2011-07-12 00:00:00 
offset days from the test date 30 Days


## Index setting to the date

In [13]:
date_time=data.set_index(data['InvoiceDate'])

## splitting data for the test before and after

In [14]:
date_time1=date_time[first_date:date_check]
date_time2=date_time[date_check:last_date]

# Z-test

select the numerical columns from this Data Frame times from this list 

In [15]:
test_list=date_time.select_dtypes(include=['float64','int64']).columns.to_list()

In [16]:
z_test_list=['Z-score','P-value']

In [17]:
from statsmodels.stats.weightstats import ztest as ztest
z_scores=ztest(date_time1[test_list],date_time2[test_list])
z_scores=pd.DataFrame(data=z_scores,index=z_test_list,columns=test_list).T
z_scores['Is Significant']=np.where(z_scores['P-value']<0.05,True,False)
z_scores=z_scores.T
z_scores

,Quantity,UnitPrice,CustomerID,Total_price
Z-score,2.093057,2.225961,NaN,2.759452
P-value,0.036344,0.026017,NaN,0.00579
Is Significant,True,True,False,True


If the p-value is lower than 0.05 this means that this value is significant 

the z-score is how many standard deviations the changed happened in a positive or a negative   